In [2]:
html = 'SmartTable All instances of Pathways in Escherichia coli K-12 substr. MG1655.html'

# index is ghetto pass by ref
def cut(string:str, start, end, index=None):
    # string = line if altStr == None else altStr
    # print(index)
    i = 0 if index is None else index[0]
    s = string.index(start, i)
    e = len(string) if end is None else string.index(end, s+len(start)) + len(end)
    if not index is None: index[0] = e
    return string[s:e]

In [3]:
raw = open(html, 'r')

# find start
while True:
    line = raw.readline()
    if line.startswith('<tr id="id62'):
        break

rows = []

# parse lines
while True:
    # print(line)

    line_i = [0]
    next = lambda: cut(line, '<td', '</td>', line_i)

    next() # checkbox
    next() # pathway link

    rows.append({
        'genes': next(),
        'id': next(),
        'name': next(),
        'reactions': next()
    })

    line = raw.readline()
    if line.startswith('</tbody></table>'):
        break

raw.close()
print(len(rows), 'rows')

442 rows


In [26]:
def rectify_html(str: str):
    return (
        str
        .replace('&gt;', '>')
        .replace('&lt;', '<')
        .replace('\\\\&#39;', "'")
        .replace('\\&#39;', "'")
        .replace('&#39;', "'")
        .replace('<SUB>', '_(').replace('</SUB>', ')')
        .replace('<SUP>', '^(').replace('</SUP>', ')')
        .replace('<sub>', '_(').replace('</sub>', ')')
        .replace('<sup>', '^(').replace('</sup>', ')')
        .replace('<i>', '').replace('</i>', '')
        .replace('<I>', '').replace('</I>', '')
    )

theone = False
pathways = {}
metabolites = set()

rc = 0 # row number for debug
for row in rows:
    # rc += 1
    # if rc != 23: 
    #     continue
    genes, id, name, rxn = row.values()

    # ====================
    # genes
    # ====================
    o_genes = []

    genes = rectify_html(genes)
    gi = [0]
    while True:
        try:
            g_raw = cut(genes, '<a', '</a>', gi)
        except ValueError:
            break

        ii = [0]
        next = lambda: cut(g_raw, '<b>', '<br>', ii)
        def get(keys):
            if not isinstance(keys, list):
                keys = [keys]

            for x in keys:
                if f"<b>{x}:</b>" in g_raw:
                    return next()

            # else
            return None

        gene = []
        keys = [['Gene', 'Pseudogene'], 'Synonyms', 'Location', 'Product']
        parsers = [
            lambda tok: list(filter(lambda x: x!='', tok[tok.index('</b>')+4:-4].strip().split(' '))), # name
            lambda tok: tok[16:-4].strip().split(', '), # synonyms
            lambda tok: tok[16:-4].strip(), # loc
            lambda tok: tok[15:-4].strip(), # product
        ]
        for k, parser in zip(keys, parsers):
            g = get(k)
            if g is not None: g = parser(g)
            gene.append(g)
        o_genes.append(tuple(gene))

        # for p in gene: print(p)
        # print()

    # ====================
    # id
    # ====================
    o_id = cut(id, '>', '<')[1:-1]
    # print(o_id)
    # theone = 'FUCCAT-PWY' == o_id

    # ====================
    # name
    # ====================
    o_name = cut(name, '>', '<')[1:-1]
    # print(o_name)

    # ====================
    # reaction
    # ====================
    o_rxn = []

    rxn = rectify_html(rxn)
    ri = [0]
    while True:
        try:
            r_raw = cut(rxn, '<a href', "WIDTH", ri)
        except ValueError:
            break

        ii = [0]
        ids = cut(r_raw, '<b>', '<br>', ii).split('<b>')
        def parse_reaction(r:str):
            dirs = [' → ', ' ↔ ', ' ← ']
            dir = None
            for d in dirs:
                if d in r:
                    dir = d
                    break

            if dir is None:
                raise Exception(f'unknown reaction direction for [{r}]')

            sides = []
            for side in r.split(dir):
                p = [s.strip() for s in side.split(' + ')]
                for par in p: metabolites.add(par)
                sides.append(p)

            if len(sides) != 2:
                print(o_id, '!!! unknown reaction format', r)
            a, b = sides
            return a, dir.strip(), b

        try:
            p_raw = cut(r_raw, '<br>', '<br>')[4:-4]
            enz = True
        except ValueError:
            p_raw = cut(r_raw, '<br>', None)[4:-8]
            enz = False
        participants = parse_reaction(p_raw)
        if enz:
            enzymes = list(map(lambda x: x.strip(),
                cut(r_raw, '<b>', 'WIDTH', ii)[15:-8]
                .split('<br>')))
        else:
            enzymes = None

        ii[0] -= 4
        reaction = (
            ids[1][17:-1].split('/'),
            ids[2][8:-4],
            participants,
            enzymes,
        )
        o_rxn.append(reaction)
        
        # for r in reaction: print(r)
        # print()

    if o_id in pathways: raise Exception(f'duplicate id [{o_id}]')
    pathways[o_id] = (
        o_name,
        o_genes,
        o_rxn,
    )

print(len(pathways), 'pathways')
print(len(metabolites), 'metabolites')

442 pathways
1412 metabolites


In [27]:
import numpy as np

folder = 'k-12_mg1655'
np.save(f'{folder}/pathways', pathways)
np.save(f'{folder}/metabolites', metabolites)

In [6]:
import json

docs = {
    'notes': [
        'unless otherwise stated dictionaries within this schema are ACTUALLY TUPLES!',
        'italics and formatting were removed',
        '[_(text)] indicate subscripts as in H_(2)O for water',
        '[^(text)] indicate superscripts as in H^(+) for a proton',
        'the prime symbol [\'], as in x\', had to be converted from HTML [&#39;] and is very likely correct, but not checked'
    ],
    'pathways.npy (dictionary)': {
        'id': {
            'name': 'string',
            'genes': [
                {
                    'identifiers': ['name', 'other databse IDs'],
                    'synonyms': ['string'],
                    'location': 'string',
                    'product': 'string',
                }
            ],
            'reactions': {
                'enzyme commission (EC) numbers': ['string'],
                'reaction ID': 'string',
                'reaction equation': {
                    'left side' : ['metabolite names'],
                    'direction': 'one of: →, ↔, ←',
                    'right side': ['metabolite names'],
                },
                'enzymes': ['enzyme names'],
            }
        }
    },
    'metabolites.npy (set)': {
        'metabolite name': 'string'
    }
}

docs = json.dumps(docs, indent=3, ensure_ascii=False)
with open(f'{folder}/docs.json', 'w') as doc_file:
    doc_file.write(docs)    